In [1]:
import os
from datetime import datetime
import glob
from collections import Counter
import pandas as pd
from typing import List, Counter
from tqdm import tqdm
import spacy
nlp = spacy.load('en_core_web_sm')
pd.set_option('display.max_rows', None)

### Variables

In [2]:
#Paths
EUROPARL = 'C:/Users/jseal/Dev/dissertation/Data/EUROPARL/'
#ENG = os.path.join(EUROPARL, 'english')
WHD = 'C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/'
WHD_CTS = 'C:/Users/jseal/Dev/dissertation/Data/WHD_CTS/'

#Data
whd_df = pd.read_csv(WHD + 'WikipediaHomographData.csv')
europarl_whd_cts_df = whd_df.drop_duplicates(subset='homograph')

### Functions

In [3]:
def get_tokens(PATH : str) -> List: 
    tokens = []
    nlp_file = nlp(' '.join(i), disable=['parser', 'tagger', 'ner']) 
    tokens = [token.lower_ for token in nlp_file if not token.is_punct]
    return tokens

In [4]:
def get_whd_subset(token_ctr : Counter, europarl_whd_cts_df : pd.DataFrame) -> pd.DataFrame:
    europarl_whd_cts_df['cts'] = europarl_whd_cts_df['homograph'].apply(lambda hg : token_ctr[hg])
    europarl_whd_cts_df = europarl_whd_cts_df[['homograph', 'cts']]
    return europarl_whd_cts_df

## Script

10-21-20 Need to check if these English texts are repeated

In [5]:
ttl_cts = []
os.chdir(EUROPARL)
for f in glob.iglob('*.en'):
    print(f)
    en_file = open(f, 'r', encoding='UTF-8').read().split(' ')
    n = 100000 #max allowed by spacy
    x = [en_file[i:i + n] for i in range(0, len(en_file), n)]
    tokens = []
    for i in tqdm(x):
        #Obtain lists of EUROPARL graphemes
        tokens.append(get_tokens("./" + f))
    tokens =  [item for sublist in tokens for item in sublist]
    token_ctr = Counter(tokens)
    #Get subsets of Europarl graphemes also in Wikipedia Homograph Data with instance counts
    cts_df = get_whd_subset(token_ctr, europarl_whd_cts_df)

    #Serialize
    DATETIME = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    
    cts_df.to_csv(os.path.join(WHD_CTS, 'europarl_whd_{}_{}_cts.csv'.format(DATETIME, f[-8:-3])))

    #ttl_cts.append(all_cts_df)     
    
#Serialize
#all_cts_df = pd.concat(ttl_cts)
DATETIME = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
#all_cts_df.to_csv(os.path.join(WHD_CTS, 'europarl_whd_{}_all_cts.csv'.format(DATETIME)))

europarl-v7.fr-en.en


100%|████████████████████████████████████████████████████████████████████████████████| 483/483 [01:55<00:00,  4.19it/s]
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
#Review data
#cts_df = all_cts_df
#Ct percents (~% Wikipedia graphemes have no instances in NXT SWBD; % have 1 instance)
#print(cts_df['cts'].value_counts(normalize=True) * 100)

#Tokens with 40 greatest count values
cts_df.sort_values(by=['cts'], ascending=False).head(40)

#Tokens with 0 instances  (graphemes, ~% of WHs)
zero_instances = cts_df[cts_df['cts'] == 0]
print(zero_instances)
print(zero_instances.shape)
print(zero_instances.shape[0]/europarl_whd_cts_df.shape[0] * 100)
#Tokens with 1-9 instances (graphemes, ~% of WHs)
single_digit_instances = cts_df[(cts_df['cts'] > 0) & (cts_df['cts'] < 10)].sort_values(by=['cts'], ascending=False)
print(single_digit_instances)
print(single_digit_instances.shape)
print(single_digit_instances.shape[0]/europarl_whd_cts_df.shape[0] *100)
#Tokens with 10 or more instances (graphemes, ~% of WHs)
more_instances = cts_df[cts_df['cts'] > 9].sort_values(by=['cts'], ascending=False)
print(more_instances)
print(more_instances.shape)
print(more_instances.shape[0]/europarl_whd_cts_df.shape[0] *100)

     homograph  cts
22     animate    0
165    incense    0
199   laminate    0
224     nestle    0
262      ravel    0
280  rerelease    0
286      rodeo    0
(7, 2)
4.320987654320987
     homograph  cts
246  postulate    9
185   intrigue    8
304  syndicate    8
12   affiliate    7
284     retard    7
59    compress    5
71   conscript    4
171  increment    4
242    pigment    4
75     consort    3
189     invert    3
69   conjugate    2
228   ornament    2
(13, 2)
8.024691358024691
        homograph    cts
318           use  34055
155         house  29143
252       present  21402
256      progress  21014
296       subject  20354
312     transport  19509
169      increase  17760
157        impact  11848
24    appropriate  11839
201          lead   9867
258       project   9446
51          close   8200
197           job   8053
177    instrument   8006
65       conflict   7822
264       reading   7533
161     implement   7149
205          live   6816
123      document   6587
81       

# Token-level alignment

In [37]:
from nltk.translate import AlignedSent

In [35]:
!pip install nltk.align

ERROR: Could not find a version that satisfies the requirement nltk.align (from versions: none)
ERROR: No matching distribution found for nltk.align


In [43]:
for f in glob.iglob('*.en'):
    en_lines = open(f, 'r', encoding='UTF-8').readlines()

for f in glob.iglob('*.fr'): 
    fr_lines = open(f, 'r', encoding="UTF-8").readlines()

en_nlp_lines = []
for e in en_lines[:5]:
    nlp_line = [token.text for token in nlp(e) if token.text != '\n']
    en_nlp_lines.append(nlp_line)    

fr_nlp_lines = []
for e in fr_lines[:5]:
    nlp_line = [token.text for token in nlp(e) if token.text != '\n']
    fr_nlp_lines.append(nlp_line)    

fren_lines = zip(en_nlp_lines, fr_nlp_lines)

In [44]:
# for e in fren_lines: 
#     print(e)

In [45]:
for e in fren_lines:
    algnsent = AlignedSent(e[0], e[1])
    print(algnsent.words)
    print(algnsent.mots)
    print(algnsent.alignment)

['Resumption', 'of', 'the', 'session']
['Reprise', 'de', 'la', 'session']

['I', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'European', 'Parliament', 'adjourned', 'on', 'Friday', '17', 'December', '1999', ',', 'and', 'I', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period', '.']
['Je', 'déclare', 'reprise', 'la', 'session', 'du', 'Parlement', 'européen', 'qui', 'avait', 'été', 'interrompue', 'le', 'vendredi', '17', 'décembre', 'dernier', 'et', 'je', 'vous', 'renouvelle', 'tous', 'mes', 'vux', 'en', 'espérant', 'que', 'vous', 'avez', 'passé', 'de', 'bonnes', 'vacances', '.']

['Although', ',', 'as', 'you', 'will', 'have', 'seen', ',', 'the', 'dreaded', "'", 'millennium', 'bug', "'", 'failed', 'to', 'materialise', ',', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were',